In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import Image
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization

In [ ]:
directory = 'dataset'
batch_size = 32
img_height = 200
img_width = 200

In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(directory,
                                                        validation_split=0.2,
                                                        subset="training",
                                                        seed=123,
                                                        color_mode='rgb',
                                                        image_size=(img_height, img_width),
                                                        batch_size=batch_size)

In [ ]:
val_data = tf.keras.utils.image_dataset_from_directory(directory,
                                                       validation_split=0.2,
                                                       subset="validation",
                                                       seed=123,
                                                       image_size=(img_height, img_width),
                                                       batch_size=batch_size)

In [ ]:
class_names = train_data.class_names

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

normalized_train = train_data.map(tf.autograph.experimental.do_not_convert(lambda x, y: (normalization_layer(x), y)))
normalized_val = val_data.map(tf.autograph.experimental.do_not_convert(lambda x, y: (normalization_layer(x), y)))

image_batch, labels_batch = next(iter(normalized_train))

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = normalized_train.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = normalized_val.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = len(class_names)

model = tf.keras.Sequential([
    Conv2D(16, 3, padding='same'),
    Activation('relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same'),
    Activation('relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same'),
    Activation('relu'),
    MaxPooling2D(),
    Dropout(0.2),
    BatchNormalization(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes)
])

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
epochs = 15
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
model.save('config/model')

In [ ]:
files = glob.glob(f'img_test/*')
for file in files:

    img = tf.keras.utils.load_img(file, target_size=(img_height, img_width))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array = normalization_layer(img_array)
    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])
    print(
        "This image most likely belongs to {} with a {:.2f} percent confidence."
            .format(class_names[np.argmax(score)], 100 * np.max(score))
    )
    pil_img = Image(file)
    display(pil_img)